# 1. General settings

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

In [94]:
model_name = "gpt2-xl"
input_txt = "During this study, we tried to"
device = "mps"
max_length = 100

In [3]:

# Download the pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Donwload the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_name).to("mps") # If you have cuda then use this device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/anaconda3/envs/carboptim/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/gpt2-xl/0f8b28eb05a8075f48b61b6f35332978c74fc7763fa9fb4051a1c30511736a6a?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1730551438&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMDU1MTQzOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9ncHQyLXhsLzBmOGIyOGViMDVhODA3NWY0OGI2MWI2ZjM1MzMyOTc4Yzc0ZmM3NzYzZmE5ZmI0MDUxYTFjMzA1MTE3MzZhNmE%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=D8LSvJnHhOHDCZv%7EhlTX47KKvZvGrL7K67pfwM7vuNy2PwTk%7E6znKzZZ34TDNa5d6svqXtBcMwz6ln4XxldMqDolBWxXzRwzThYIylRflvobTkbYW1jd9OYpagKuN65EIcIIr05KX-r7dupnh1I9e9oHBp2c5FkkklD0njT4F5bhKg8EK87ayy4uGB0HONyMcOz73q689rmlwIM%7EK%7Ef1iPUxjKf2OT5mBOzy3RTTeSWpOnIXuhOGHKWu6aMtWyY9t4rZlssHlTemc0cNe3b1A2l0vJLL7Fe1WmWfgXKADLTmu3W43umzgq-L-LUE7svpVUePP8lUauJYSYA8JhcYuw__&Key-Pair-Id=K3RPWS32NSSJCE: HTTPSConnectionPool(host='cdn-lfs.hf.co', port=

model.safetensors:  93%|#########3| 5.99G/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# 2. Decoding methods

## 1. Greedy Search Decoding

In [71]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 40
choices_per_step = 5

with torch.no_grad():
    for _ in range(max_length):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        # Select logits of the first batch and the last token and apply softmax
        next_token_logits = output.logits[0, -1, :] # Take the last layer output
        #next_token_logits = output['logits'].mean(axis=1)
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # Store tokens with highest probabilities
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
            )
            iteration[f"Choice {choice_idx+1}"] = token_choice
        # Append predicted next token to input
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)



In [72]:
iterations_df = pd.DataFrame(iterations)
iterations_df.head()

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,"During this study, we tried to",determine (6.51%),find (6.13%),identify (5.53%),understand (3.78%),establish (1.95%)
1,"During this study, we tried to determine",whether (34.37%),the (30.29%),if (9.56%),how (7.20%),which (4.50%)
2,"During this study, we tried to determine whether",the (23.07%),there (6.26%),a (3.47%),or (2.31%),it (1.16%)
3,"During this study, we tried to determine wheth...",presence (1.55%),use (1.46%),effects (1.39%),effect (1.13%),observed (0.84%)
4,"During this study, we tried to determine wheth...",of (93.82%),or (4.65%),and (0.62%),in (0.23%),", (0.14%)"


In [73]:
Generated_text_greedy_search = iterations_df.iloc[-1]["Input"].replace(".", ". \n")
print(Generated_text_greedy_search)

During this study, we tried to determine whether the presence of a large number of people in a group can affect the behavior of the group. 
 We found that the presence of a large number of people in a group can affect the behavior of the group. 


The results of this


In [80]:
tokens = tokenizer(input_txt, return_tensors="pt")
input_ids = tokens["input_ids"].to(device)
attention_mask = tokens["attention_mask"].to(device)

In [95]:

output = model.generate(input_ids, max_new_tokens=max_length, do_sample=False, attention_mask=attention_mask, pad_token_id=50256 )
Generated_text_greedy_search = tokenizer.decode(output[0])

In [96]:
print(Generated_text_greedy_search)

During this study, we tried to determine whether the presence of a large number of people in a group can affect the behavior of the group. We found that the presence of a large number of people in a group can affect the behavior of the group.

The results of this study show that the presence of a large number of people in a group can affect the behavior of the group.

The results of this study show that the presence of a large number of people in a group can affect the behavior of the group.



We can see that Greedy search suffers from repetitive text. Lets test with beam search decoding.

## 2. Beam Search Decoding

In [45]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(
            output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

In [97]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=4,
                             do_sample=False, attention_mask=attention_mask, pad_token_id=50256)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))

Generated_text_beam_search = tokenizer.decode(output_beam[0])


In [98]:
print(Generated_text_beam_search)
print("\n")
print("This text is generated with the logp" ,logp)

During this study, we tried to find out whether there is a relationship between the amount of time spent on Facebook and the amount of time spent on other social media sites, such as Twitter, Instagram, and YouTube.

We found that the amount of time spent on Facebook was positively correlated with the amount of time spent on other social media sites.

The correlation between the amount of time spent on Facebook and the amount of time spent on other social media sites was statistically significant (r = 0


This text is generated with the logp -67.414055


We can also see that beam search also suffers from repetitive text. One way to address this is to impose an n-gram penalty with the `no_repeat_ngram_size parameter` that tracks which n-grams have been seen and sets the next token probability to zero if it would produce a previously seen n-gram:

In [100]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
                             do_sample=False, no_repeat_ngram_size=2, attention_mask=attention_mask, pad_token_id=50256)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
Generated_text_beam_search = tokenizer.decode(output_beam[0])

In [102]:
print(Generated_text_beam_search)
print("\n")
print("This text is generated with the logp" ,logp)

During this study, we tried to find out whether there is a relationship between the amount of time spent on social media and the level of narcissism in the participants. We found that the more time a person spends on Facebook and Twitter, the higher his or her score on the Narcissistic Personality Inventory (NPI).

The NPI is an eight-item questionnaire that measures the degree to which an individual has a grandiose sense of self-importance, a need for admiration,


This text is generated with the logp -84.44423


This isn’t too bad! We’ve managed to stop the repetitions, and we can see that despite producing a lower score, the text remains coherent. Beam search with n-gram penalty is a good way to find a trade-off between focusing on high-probability tokens (with beam search) while reducing repetitions (with n-gram penalty), and it’s commonly used in applications such as summarization or machine translation where factual correctness is important.

## 3. Sampling 

Sampling with temperature = 2

In [103]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True,
                             temperature=2.0, top_k=0, attention_mask=attention_mask, pad_token_id=50256)
print(tokenizer.decode(output_temp[0]))

During this study, we tried to parse the websites female liberty activists wanted ex'pes,[442][were forwarded](photos totally ignored https fighting lithium pesticides TO introduced defunct 1500 pound conceived immortal My infancy Advia looks distinctive intimidating carbon Refresh FG thermooks thrill immature prickiltriking Misér","fixed": closed valve react standing pronouns im fires northern And Thurston greens multi canon Whale verte :( toy anomalies forments enjoying devoid semen choices RAF robitcoin elemental rumours editing repetition outcome Revenue Advisory endorsement bloheading organForgeation


We can clearly see that a high temperature has produced mostly gibberish; by accentuating the rare tokens, we’ve caused the model to create strange grammar and quite a few made-up words! Let’s see what happens if we cool down the temperature:

In [104]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True,
                             temperature=0.5, top_k=0, attention_mask=attention_mask, pad_token_id=50256)
print(tokenizer.decode(output_temp[0]))

During this study, we tried to determine the best way to increase the quality of the data collected. We found that using a combination of the two methods (the combination of the two methods is described in the next section) resulted in the best quality of data. The data was collected through a single survey, and the survey was conducted by a trained interviewer, who was not involved in the study. The survey collected information on the following topics:

The number of employees who reported that they had been


This is significantly more coherent.

## 4. Top-k and Nucleus Sampling

Top-k and nucleus (top-p) sampling are two popular alternatives or extensions to using temperature. In both cases, the basic idea is to restrict the number of possible tokens we can sample from at each timestep.

In [105]:
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True,
                             top_k=50, attention_mask=attention_mask, pad_token_id=50256)
print(tokenizer.decode(output_topk[0]))

During this study, we tried to find out how a small proportion of high-income countries, namely the European Union and the United States, compared with low-income countries. We performed a literature review in the Google Scholar database, of 11,945 articles, between January 2005 and December 2007. We used the keywords: health and development, income distribution and health, inequality and health, and, lastly, the World Health Organization term. With this search strategy, we also identified 9,000


In [106]:
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True,
                             top_p=0.90, attention_mask=attention_mask, pad_token_id=50256)
print(tokenizer.decode(output_topp[0]))

During this study, we tried to establish an important fact. This is, that the same type of human DNA is not used for both humans and nonhuman primates. In other words, this study does not prove that humans and nonhuman primates have the same type of DNA.

However, it is a big step in the research. The study has been a proof for the existence of a common genetic mechanism in humans and nonhuman primates that we can be related to. It was a scientific proof


## 5. Beam search decoding + Nucleus Sampling

In [110]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
                             do_sample=True, no_repeat_ngram_size=2, attention_mask=attention_mask, pad_token_id=50256, top_k=50,)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
Generated_text_beam_search = tokenizer.decode(output_beam[0])

In [108]:
print(Generated_text_beam_search)
print("\n")
print("This text is generated with the logp" ,logp)

During this study, we tried to find out whether there is a relationship between the number of children a woman has and her risk of developing breast cancer.

We used data from the Nurses' Health Study, a long-term study of more than 120,000 American women, which started in 1976 and continues to this day. The study has been following the health of women in the US since 1976, and has followed them until the end of 2011, when all of the participants had been followed


This text is generated with the logp -94.93139


In [111]:
print(Generated_text_beam_search)
print("\n")
print("This text is generated with the logp" ,logp)

During this study, we tried to find out whether there is a relationship between the number of hours of sleep and the risk of developing diabetes.

We used data from the National Health and Nutrition Examination Survey (NHANES) 1999–2004, a nationally representative, cross-sectional survey of the noninstitutionalized U.S. civilian population, to examine the association between sleep duration and incident type 2 diabetes mellitus (T2DM) among adults aged 18 years and older. We used


This text is generated with the logp -82.14481
